In [1]:
import pandas as pd
import numpy as np
#importamos la funcion rankdata de scipy
from scipy.stats import rankdata


# DATASETS TFM

In [2]:
#cargamos el dataset .xlsx
df = pd.read_excel('./20240626_154803_CCR_GridSearch+AMAE_bien.xlsx')


In [3]:
# Extraemos del dataset las columnas dataset, estimator_name, random_state, QWK, MAE, CCR, AMAE, MMAE
df_filter = df[['dataset', 'estimator_name', 'random_state', 'QWK', 'MAE', 'CCR', 'AMAE', 'MMAE']]
#ordenamos el dataset por  random_state, dataset, estimator_name
df_filter = df_filter.sort_values(by=['random_state', 'dataset', 'estimator_name'])
#reseteamos el indice
df_filter = df_filter.reset_index(drop=True)

#Mostramos el numero de dataset unicos
print("Numero de datasets: ",df_filter['dataset'].nunique())

#Mostramos el numero de estimadores unicos
print("Numero de estimadores: ",df_filter['estimator_name'].nunique())


#mostamos el dataset ordenado
print(df_filter)

Numero de datasets:  15
Numero de estimadores:  5
              dataset      estimator_name  random_state       QWK     MAE  \
0                 ERA          logisticat             0  0.585819  1.2320   
1                 ERA  logisticat_desb_v2             0  0.586797  1.4880   
2                 ERA          logisticit             0  0.366007  2.8400   
3                 ERA  logisticit_desb_v2             0  0.366768  2.8160   
4                 ERA   logisticregressor             0  0.482570  1.8920   
...               ...                 ...           ...       ...     ...   
2245  winequality-red          logisticat            29  0.510413  0.4475   
2246  winequality-red  logisticat_desb_v2            29  0.308933  1.4950   
2247  winequality-red          logisticit            29  0.487230  0.4600   
2248  winequality-red  logisticit_desb_v2            29  0.323757  1.8525   
2249  winequality-red   logisticregressor            29  0.483435  0.7950   

         CCR      AMAE   

In [4]:
# Cada 30 filas (que corresponde a un dataset) le calaculamos el rankdata a las columnas MAE y MMAE
# y lo guardamos en las columnas 'rank_MAE' y 'rank_MMAE' respectivamente
for i in range(0, len(df), df_filter['estimator_name'].nunique()):
    df_filter.loc[i:i+df_filter['estimator_name'].nunique()-1, 'rank_MAE'] = rankdata(df_filter.loc[i:i+df_filter['estimator_name'].nunique()-1, 'MAE'])
    df_filter.loc[i:i+df_filter['estimator_name'].nunique()-1, 'rank_MMAE'] = rankdata(df.loc[i:i+df_filter['estimator_name'].nunique()-1, 'MMAE'])


In [5]:
print(df_filter)

              dataset      estimator_name  random_state       QWK     MAE  \
0                 ERA          logisticat             0  0.585819  1.2320   
1                 ERA  logisticat_desb_v2             0  0.586797  1.4880   
2                 ERA          logisticit             0  0.366007  2.8400   
3                 ERA  logisticit_desb_v2             0  0.366768  2.8160   
4                 ERA   logisticregressor             0  0.482570  1.8920   
...               ...                 ...           ...       ...     ...   
2245  winequality-red          logisticat            29  0.510413  0.4475   
2246  winequality-red  logisticat_desb_v2            29  0.308933  1.4950   
2247  winequality-red          logisticit            29  0.487230  0.4600   
2248  winequality-red  logisticit_desb_v2            29  0.323757  1.8525   
2249  winequality-red   logisticregressor            29  0.483435  0.7950   

         CCR      AMAE      MMAE  rank_MAE  rank_MMAE  
0     0.2600  1.411

In [6]:
# Ahora debemos sacar la media de los rankings obtenidos en el paso anterior
# para cada dataset y para cada estimador
# para ello agrupamos por dataset y estimator_name y calculamos la media de rank_MAE y rank_MMAE
df_filter['mean_rank_MAE'] = df_filter.groupby(['dataset', 'estimator_name'])['rank_MAE'].transform('mean')
df_filter['mean_rank_MMAE'] = df_filter.groupby(['dataset', 'estimator_name'])['rank_MMAE'].transform('mean')

print(df_filter)

              dataset      estimator_name  random_state       QWK     MAE  \
0                 ERA          logisticat             0  0.585819  1.2320   
1                 ERA  logisticat_desb_v2             0  0.586797  1.4880   
2                 ERA          logisticit             0  0.366007  2.8400   
3                 ERA  logisticit_desb_v2             0  0.366768  2.8160   
4                 ERA   logisticregressor             0  0.482570  1.8920   
...               ...                 ...           ...       ...     ...   
2245  winequality-red          logisticat            29  0.510413  0.4475   
2246  winequality-red  logisticat_desb_v2            29  0.308933  1.4950   
2247  winequality-red          logisticit            29  0.487230  0.4600   
2248  winequality-red  logisticit_desb_v2            29  0.323757  1.8525   
2249  winequality-red   logisticregressor            29  0.483435  0.7950   

         CCR      AMAE      MMAE  rank_MAE  rank_MMAE  mean_rank_MAE  \
0  

In [7]:
# Mostramos una tabla reducida que contenga dataset, estimator_name, mean_rank_MAE, mean_rank_MMAE
df_filter2 = df_filter[['dataset', 'estimator_name', 'mean_rank_MAE', 'mean_rank_MMAE']]
df_filter2 = df_filter2.drop_duplicates()
print(df_filter2)

            dataset      estimator_name  mean_rank_MAE  mean_rank_MMAE
0               ERA          logisticat       1.000000        3.300000
1               ERA  logisticat_desb_v2       2.000000        2.683333
2               ERA          logisticit       4.333333        2.666667
3               ERA  logisticit_desb_v2       4.666667        3.083333
4               ERA   logisticregressor       3.000000        3.266667
..              ...                 ...            ...             ...
70  winequality-red          logisticat       1.450000        2.983333
71  winequality-red  logisticat_desb_v2       4.000000        2.900000
72  winequality-red          logisticit       1.550000        2.833333
73  winequality-red  logisticit_desb_v2       5.000000        3.100000
74  winequality-red   logisticregressor       3.000000        3.183333

[75 rows x 4 columns]


In [8]:
#Por ultimo extraemos cada ranking del df por dataset y lo guardamos en un archivo .xlsx donde cada pagina corresponde a un dataset

with pd.ExcelWriter('rankings.xlsx') as writer:
    for dataset in df_filter['dataset'].unique():
        df_filter3 = df_filter[df_filter['dataset'] == dataset]
        # Usa el nombre del dataset como nombre de la hoja
        df_filter3.to_excel(writer, sheet_name=dataset, index=False)


In [9]:
#combinamos los rankings en un solo archivo 
df_filter.to_excel('./*_rankings.xlsx', index=False)

# Dataset REVIEW

In [10]:
################################
# Rutas, Info y Carga de Datos #
################################
import glob

ruta= "../Review/Ordinal/*.csv"
print("Leyendo .csv de la ruta: ", ruta, "...\n")
#leemos todos los archivos csv de la carpeta Ordinal y los guardamos en una lista de dataframes
df_list= [pd.read_csv(f,header=1) for f in glob.glob(ruta)]
#Creamos una lista con el nombre de los archivos csv y extraemos el nombre de los métodos
nombres= [f for f in glob.glob(ruta)]
metodos = [name.split("Results-")[1].split(".csv")[0] for name in nombres]
#Lista de Dataset
dataset_list=['contact-lenses', 'pasture', 'squash-stored', 'squash-unstored', 'tae', 'newthyroid', 'balance-scale', 'SWD', 'car', 'bondrate', 'toy', 'eucalyptus', 'LEV', 'automobile', 'winequality-red', 'ESL', 'ERA']


#Tabla final con los rankings
#df_final = pd.DataFrame(columns=['dataset', 'estimator_name', 'random_state', 'MAE', 'MZE'])
print("Dataframes cargados correctamente: ", metodos, "\n")

Leyendo .csv de la ruta:  ../Review/Ordinal/*.csv ...

Dataframes cargados correctamente:  ['POM', 'SVORIMLin'] 



In [11]:
for df, metodo in zip(df_list, metodos):
    print("Procesando: ", metodo, "...\n")
    """
    Se ha colado la primera columna de los df como el indice luego hay que arreglar esto:
    """
    df.reset_index(inplace=True)
    #corremos el encabezado una posicion a la izquierda
    encabezados=list(df.columns)
    #eliminamos el primer elemento
    encabezados.pop(0)
    #eliminamos ultima columna de df
    df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
    df.columns=encabezados

    #Añadimos la columna "random_state" al df cuyo valor sera el indice de la fila
    df['random_state']=df.index

Procesando:  POM ...

Procesando:  SVORIMLin ...



In [12]:
df_list[0]

,MZE,MAE,Time,MZE.1,MAE.1,Time.1,MZE.2,MAE.2,Time.2,MZE.3,...,MZE.14,MAE.14,Time.14,MZE.15,MAE.15,Time.15,MZE.16,MAE.16,Time.16,random_state
0,0.166667,0.333333,0.277720,0.666667,0.777778,0.398996,0.692308,0.923077,0.561309,0.769231,...,0.4075,0.4450,0.700013,0.319672,0.352459,0.525073,0.728,1.172,0.476681,0
1,0.333333,0.666667,0.301020,0.666667,0.777778,0.390884,0.461538,0.615385,0.650275,0.615385,...,0.4025,0.4325,0.770109,0.319672,0.327869,0.528113,0.772,1.248,0.481841,1
2,0.333333,0.333333,0.290709,0.444444,0.555556,0.469870,0.538462,0.538462,0.612259,0.692308,...,0.4125,0.4450,0.709634,0.319672,0.319672,0.435674,0.708,1.144,0.510575,2
3,0.166667,0.333333,0.291362,0.333333,0.333333,0.389949,0.692308,1.000000,0.619493,0.692308,...,0.4000,0.4400,0.783432,0.270492,0.270492,0.509698,0.784,1.328,0.479516,3
4,0.333333,0.333333,0.304676,0.444444,0.555556,0.398925,0.384615,0.461538,0.582329,0.615385,...,0.4025,0.4300,0.784082,0.270492,0.286885,0.549291,0.724,1.144,0.460644,4
5,0.333333,0.333333,0.278755,0.555556,0.555556,0.396986,0.615385,0.769231,0.597459,0.923077,...,0.3850,0.4200,0.706131,0.188525,0.213115,0.510498,0.740,1.148,0.474808,5
6,0.666667,0.666667,0.289150,0.333333,0.333333,0.401645,0.692308,0.923077,0.597559,0.923077,...,0.4375,0.4775,0.770898,0.254098,0.270492,0.544577,0.756,1.260,0.491665,6
7,0.166667,0.166667,0.239458,0.333333,0.333333,0.481985,0.307692,0.384615,0.596172,0.846154,...,0.3825,0.4150,0.807898,0.295082,0.311475,0.528829,0.732,1.168,0.478542,7
8,0.166667,0.333333,0.276851,0.333333,0.333333,0.427165,0.615385,0.769231,0.640166,0.538462,...,0.3900,0.4325,0.734836,0.270492,0.270492,0.514071,0.716,1.224,0.493183,8
9,0.333333,0.500000,0.311873,0.666667,0.666667,0.398626,0.384615,0.461538,0.561322,0.692308,...,0.4075,0.4425,0.766264,0.327869,0.352459,0.550818,0.752,1.256,0.479732,9


In [14]:
df_final = pd.DataFrame(columns=['dataset', 'estimator_name', 'random_state', 'MAE', 'MZE'], index=range(1022))

#Iteramos sobre los dataframes de la lista df_list
for i in range(0, len(df_list)):
    print("Procesando: ", metodos[i], "...\n")
    #iteramos sobre los datasets
    for j in range(len(dataset_list)):
        print("Procesando dataset: ", dataset_list[j], "...\n")
        #Cada dataset ocupa 30 filas (las correspondientes a los 30 random_state)
        inicio = i*len(dataset_list)*30+j*30
        fin = inicio + 29
        #De la fila 0 a la 29 asignamos el dataset, el random_state y el metodo usado
        df_final.loc[inicio:fin, 'dataset'] = dataset_list[j]
        df_final.loc[inicio:fin, 'random_state'] = df_list[i]['random_state'].values[:30]
        df_final.loc[inicio:fin, 'estimator_name'] = metodos[i]
        
        #Asignamos las columnas MAE y MZE
        df_final.loc[inicio:fin, 'MAE'] = df_list[i].iloc[:, 3*j].values
        df_final.loc[inicio:fin, 'MZE'] = df_list[i].iloc[:, 3*j+1].values


Procesando:  POM ...

Procesando dataset:  contact-lenses ...

Procesando dataset:  pasture ...

Procesando dataset:  squash-stored ...

Procesando dataset:  squash-unstored ...

Procesando dataset:  tae ...

Procesando dataset:  newthyroid ...

Procesando dataset:  balance-scale ...

Procesando dataset:  SWD ...

Procesando dataset:  car ...

Procesando dataset:  bondrate ...

Procesando dataset:  toy ...

Procesando dataset:  eucalyptus ...

Procesando dataset:  LEV ...

Procesando dataset:  automobile ...

Procesando dataset:  winequality-red ...

Procesando dataset:  ESL ...

Procesando dataset:  ERA ...

Procesando:  SVORIMLin ...

Procesando dataset:  contact-lenses ...

Procesando dataset:  pasture ...

Procesando dataset:  squash-stored ...

Procesando dataset:  squash-unstored ...

Procesando dataset:  tae ...

Procesando dataset:  newthyroid ...

Procesando dataset:  balance-scale ...

Procesando dataset:  SWD ...

Procesando dataset:  car ...

Procesando dataset:  bondrate .

In [ ]:
print(df_final)
#exportamos el dataframe a un archivo .xlsx
df_final.to_excel('PRUEBA.xlsx', index=False)

             dataset estimator_name random_state       MAE       MZE
0     contact-lenses            POM            0  0.166667  0.333333
1     contact-lenses            POM            1  0.333333  0.666667
2     contact-lenses            POM            2  0.333333  0.333333
3     contact-lenses            POM            3  0.166667  0.333333
4     contact-lenses            POM            4  0.333333  0.333333
...              ...            ...          ...       ...       ...
1017             ERA      SVORIMLin           27     0.764       1.2
1018             ERA      SVORIMLin           28     0.748      1.24
1019             ERA      SVORIMLin           29      0.74     1.272
1020             NaN            NaN          NaN       NaN       NaN
1021             NaN            NaN          NaN       NaN       NaN

[1022 rows x 5 columns]


In [22]:
#unimos los dataframes df_filter y df_final en un solo dataframe df_union manteniendo las columnas en comun
df_union=pd.concat([df_filter, df_final], axis=0, ignore_index=True, sort=False)
#df_union = pd.merge(df_filter, df_final, on=['dataset', 'estimator_name', 'random_state'], how='inner')
print(df_union)

     dataset      estimator_name random_state       QWK    MAE    CCR  \
0        ERA          logisticat            0  0.585819  1.232  0.260   
1        ERA  logisticat_desb_v2            0  0.586797  1.488  0.252   
2        ERA          logisticit            0  0.366007   2.84  0.108   
3        ERA  logisticit_desb_v2            0  0.366768  2.816  0.108   
4        ERA   logisticregressor            0  0.482570  1.892  0.216   
...      ...                 ...          ...       ...    ...    ...   
3267     ERA           SVORIMLin           27       NaN  0.764    NaN   
3268     ERA           SVORIMLin           28       NaN  0.748    NaN   
3269     ERA           SVORIMLin           29       NaN   0.74    NaN   
3270     NaN                 NaN          NaN       NaN    NaN    NaN   
3271     NaN                 NaN          NaN       NaN    NaN    NaN   

          AMAE      MMAE  rank_MAE  rank_MMAE  mean_rank_MAE  mean_rank_MMAE  \
0     1.411905  2.000000       1.0        1

# PRUEBA UNION


In [41]:
###################
# CARGA DE DATOS  #
###################
#------------------> RESULTADOS TFM
results_tfm=pd.read_excel('./20240626_154803_CCR_GridSearch+AMAE_bien.xlsx')



#------------------> RESULTADOS REVIEW
#Ruta de los resultados del review
ruta= "../Review/Ordinal/*.csv"
#leemos todos los archivos csv de la carpeta Ordinal y los guardamos en una lista de dataframes
df_list= [pd.read_csv(f,header=1) for f in glob.glob(ruta)]
#Creamos una lista con el nombre de los archivos csv y extraemos el nombre de los métodos
nombres= [f for f in glob.glob(ruta)]
metodos = [name.split("Results-")[1].split(".csv")[0] for name in nombres]
#Lista de Dataset
#dataset_list=resultados_tfm['dataset'].unique()
dataset_list=['contact-lenses', 'pasture', 'squash-stored', 'squash-unstored', 'tae', 'newthyroid', 'balance-scale', 'SWD', 'car', 'bondrate', 'toy', 'eucalyptus', 'LEV', 'automobile', 'winequality-red', 'ESL', 'ERA']


In [42]:
# -----------------> PREPROCESAMIENTO TFM
# Extraemos del dataset las columnas necesarias
df_TFM = results_tfm[['dataset', 'estimator_name', 'random_state', 'MAE', 'CCR']]#, 'QWK', 'AMAE', 'MMAE']]
#ordenamos el dataset por  random_state, dataset, estimator_name
df_TFM = df_TFM.sort_values(by=['random_state', 'dataset', 'estimator_name'])
#reseteamos el indice
df_TFM = df_TFM.reset_index(drop=True)
#Calculamos el MZE a partir del CCR
df_TFM["MZE"]=1-df_TFM["CCR"]

In [43]:
# -----------------> PREPROCESAMIENTO REVIEW
for df, metodo in zip(df_list, metodos):
    print("Procesando: ", metodo, "...\n")
    """
    Se ha colado la primera columna de los df como el indice luego hay que arreglar esto:
    """
    df.reset_index(inplace=True)
    #corremos el encabezado una posicion a la izquierda
    encabezados=list(df.columns)
    #eliminamos el primer elemento
    encabezados.pop(0)
    #eliminamos ultima columna de df
    df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
    df.columns=encabezados

    #Añadimos la columna "random_state" al df cuyo valor sera el indice de la fila
    df['random_state']=df.index

"""
Tras tener el df en el formato correcto, vamos a crear un df_REVIEW que contenga la misma estructura que el df_TFM
"""

df_REVIEW = pd.DataFrame(columns=['dataset', 'estimator_name', 'random_state', 'MAE', 'MZE'], index=range(1022))

#Iteramos sobre los dataframes de la lista df_list
for i in range(0, len(df_list)):
    print("Procesando: ", metodos[i], "...\n")
    #iteramos sobre los datasets
    for j in range(len(dataset_list)):
        print("Procesando dataset: ", dataset_list[j], "...\n")
        #Cada dataset ocupa 30 filas (las correspondientes a los 30 random_state)
        inicio = i*len(dataset_list)*30+j*30
        fin = inicio + 29
        #De la fila 0 a la 29 asignamos el dataset, el random_state y el metodo usado
        df_REVIEW.loc[inicio:fin, 'dataset'] = dataset_list[j]
        df_REVIEW.loc[inicio:fin, 'random_state'] = df_list[i]['random_state'].values[:30]
        df_REVIEW.loc[inicio:fin, 'estimator_name'] = metodos[i]
        
        #Asignamos las columnas MAE y MZE
        df_REVIEW.loc[inicio:fin, 'MZE'] = df_list[i].iloc[:, 3*j].values
        df_REVIEW.loc[inicio:fin, 'MAE'] = df_list[i].iloc[:, 3*j+1].values

df_REVIEW['CCR']=1-df_REVIEW['MZE']

Procesando:  POM ...

Procesando:  SVORIMLin ...

Procesando:  POM ...

Procesando dataset:  contact-lenses ...

Procesando dataset:  pasture ...

Procesando dataset:  squash-stored ...

Procesando dataset:  squash-unstored ...

Procesando dataset:  tae ...

Procesando dataset:  newthyroid ...

Procesando dataset:  balance-scale ...

Procesando dataset:  SWD ...

Procesando dataset:  car ...

Procesando dataset:  bondrate ...

Procesando dataset:  toy ...

Procesando dataset:  eucalyptus ...

Procesando dataset:  LEV ...

Procesando dataset:  automobile ...

Procesando dataset:  winequality-red ...

Procesando dataset:  ESL ...

Procesando dataset:  ERA ...

Procesando:  SVORIMLin ...

Procesando dataset:  contact-lenses ...

Procesando dataset:  pasture ...

Procesando dataset:  squash-stored ...

Procesando dataset:  squash-unstored ...

Procesando dataset:  tae ...

Procesando dataset:  newthyroid ...

Procesando dataset:  balance-scale ...

Procesando dataset:  SWD ...

Procesando 

In [44]:
#-----------------------------> UNION DE DATAFRAMES
#Unimos los dos dataframes
df_union = pd.concat([df_TFM, df_REVIEW],axis=0, ignore_index=True, sort=False)
#ordenamos el dataframe por dataset, estimator_name, random_state
df_union = df_union.sort_values(by=['dataset', 'random_state', 'estimator_name'])
#reseteamos el indice
df_union = df_union.reset_index(drop=True)

In [56]:
print(df_union)
df_union.to_excel('UNION.xlsx', index=False)

              dataset      estimator_name random_state     MAE     CCR  \
0                 ERA                 POM            0  1.1720   0.272   
1                 ERA           SVORIMLin            0  1.1960   0.256   
2                 ERA          logisticat            0  1.2320    0.26   
3                 ERA  logisticat_desb_v2            0  1.4880   0.252   
4                 ERA          logisticit            0  2.8400   0.108   
...               ...                 ...          ...     ...     ...   
3267  winequality-red          logisticit           29  0.4600  0.5675   
3268  winequality-red  logisticit_desb_v2           29  1.8525  0.0875   
3269  winequality-red   logisticregressor           29  0.7950  0.4025   
3270              NaN                 NaN          NaN     NaN     NaN   
3271              NaN                 NaN          NaN     NaN     NaN   

         MZE  rank_MAE  rank_MZE  mean_rank_MAE  mean_rank_MZE  \
0     0.7280       1.0       1.0       1.9166

In [51]:
############
# RANKDATA #
############

#Asegúrate de que 'MAE' y 'MZE' son numéricos
df_union['MAE'] = pd.to_numeric(df_union['MAE'], errors='coerce')
df_union['MZE'] = pd.to_numeric(df_union['MZE'], errors='coerce')

# Realizamos el rankdata para MAE y MZE
for i in range(0, len(df_union), df_union['estimator_name'].nunique()):
    df_union.loc[i:i+df_union['estimator_name'].nunique()-1, 'rank_MAE'] = rankdata(df_union.loc[i:i+df_union['estimator_name'].nunique()-1, 'MAE'].to_numpy())
    df_union.loc[i:i+df_union['estimator_name'].nunique()-1, 'rank_MZE'] = rankdata(df_union.loc[i:i+df_union['estimator_name'].nunique()-1, 'MZE'].to_numpy())

In [53]:
#Calculamos las medias de cada ranking
df_union['mean_rank_MAE'] = df_union.groupby(['dataset', 'estimator_name'])['rank_MAE'].transform('mean')
df_union['mean_rank_MZE'] = df_union.groupby(['dataset', 'estimator_name'])['rank_MZE'].transform('mean')
#

In [55]:
#Calculamos las medias de las medias obtenidas antes segun cada estimador
df_union['mean_mean_rank_MAE'] = df_union.groupby(['estimator_name'])['mean_rank_MAE'].transform('mean')
df_union['mean-mean_rank_MZE'] = df_union.groupby(['estimator_name'])['mean_rank_MZE'].transform('mean')


In [57]:
#extraemos de df_union una tabla que contenga dataset, estimator_name, mean_rank_MAE, mean_rank_MZE
tabla1 = df_union[['dataset', 'estimator_name', 'mean_rank_MAE', 'mean_rank_MZE']]
tabla1 = tabla1.drop_duplicates()
print(tabla1)

              dataset      estimator_name  mean_rank_MAE  mean_rank_MZE
0                 ERA                 POM       1.916667       2.616667
1                 ERA           SVORIMLin       2.000000       3.400000
2                 ERA          logisticat       2.083333       1.650000
3                 ERA  logisticat_desb_v2       4.000000       2.516667
4                 ERA          logisticit       6.333333       6.550000
...               ...                 ...            ...            ...
3063  winequality-red  logisticat_desb_v2       6.000000       6.000000
3064  winequality-red          logisticit       2.466667       2.266667
3065  winequality-red  logisticit_desb_v2       7.000000       7.000000
3066  winequality-red   logisticregressor       5.000000       5.000000
3270              NaN                 NaN            NaN            NaN

[110 rows x 4 columns]


In [59]:
#extraemos una tabla que contenga estimator_name, mean_mean_rank_MAE, mean_mean_rank_MZE
tabla2 = df_union[['estimator_name', 'mean_mean_rank_MAE', 'mean-mean_rank_MZE']]
tabla2 = tabla2.drop_duplicates()
#ordenamos la tabla por mean_mean_rank_MAE
tabla2 = tabla2.sort_values(by=['mean_mean_rank_MAE'])
print(tabla2)

          estimator_name  mean_mean_rank_MAE  mean-mean_rank_MZE
1              SVORIMLin            2.874510            3.162745
2             logisticat            3.275556            3.340000
4             logisticit            4.143333            4.143333
0                    POM            4.233333            4.224510
6      logisticregressor            4.243333            3.831111
3     logisticat_desb_v2            4.378889            4.441111
5     logisticit_desb_v2            4.972222            4.941111
3270                 NaN                 NaN                 NaN


In [60]:
#guardamos  df_union, tabla1 y tabla2 en un mismo archivo .xlsx
with pd.ExcelWriter('RANKING_def.xlsx') as writer:
    df_union.to_excel(writer, sheet_name='Todo_rank', index=False)
    tabla1.to_excel(writer, sheet_name='Rank_por_dataset', index=False)
    tabla2.to_excel(writer, sheet_name='Rank_por_estimador', index=False)